In [1]:
from rembg import remove 

# PIL 패키지에서 Image 클래스 불러오기
from PIL import Image 
from tqdm import tqdm
import os
import numpy as np
import matplotlib.pyplot as plt
from os import makedirs
from os.path import join, exists, expanduser

import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tqdm import tqdm

C:\Users\jisuj\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jisuj\anaconda3\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\jisuj\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\jisuj\anaconda3\lib\site-packages\numba\np\ufunc\parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
C:\Users\jisuj\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [4]:
import firebase_admin
from firebase_admin import credentials, db
from flask import Flask, render_template, redirect, request

In [5]:
# 내 firebase key로 바꿈
cred = credentials.Certificate("hello-data-ce477-firebase-adminsdk-7cfjd-81bc9253f1.json")
firebase_admin.initialize_app(cred, {
    'databaseURL' : 'https://hello-data-ce477-default-rtdb.firebaseio.com/'})

In [55]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [ ]:
# 지수
app = Flask(__name__)

@app.route('/', methods=['GET'])

def get_data():
    #변환할 이미지 feature vector 뽑기
    image_path = 'C:\\Users\\jisuj\\Desktop\\chiwawa2.jpg'
    img = Image.open(image_path)
    out = remove(img)
    out_image = out.convert("RGB")
    out_image = out_image.resize((224,224))
    img = image.img_to_array(out_image)
    x = preprocess_input(np.expand_dims(img.copy(), axis=0))
    resnet = ResNet50(weights='imagenet')
    test_preds = resnet.predict(x)
    test_preds = test_preds.tolist()
    
    
    
    ## db의 모든 feature vector 가져오기
    ref = db.reference('User')  # 데이터를 가져올 경로 지정
    target_data = ref.get()
    # Firebase Realtime Database에서 데이터 가져오기
    user_count = 0
    if target_data:
        user_count = len(target_data)
        user_count -= 1 #왠진 모르겠지만 앞에 none 데이터 하나 추가됨
        print(user_count)
    
    
    result = []
    for i in range(user_count):
        path = 'User/'+str(i+1)
        ref =  db.reference(path)
        data = ref.get()
        if data :
            desired_data = data.get('Test5')
            print(type(desired_data))
            print("불러옴")
            b = desired_data.split(' ')
            b = list(map(float, b[:-1]))   
            result.append(b)
            print("추가함")
        
    ## db데이터와 cosine 유사도
    def cos_sim(A, B):
        return dot(A, B)/(norm(A)*norm(B))
    
    sim_list = []
    
    for i in range(len(result)):
        a =cos_sim(np.array(test_preds),np.array(result[i]))
        sim_list.append(a)
    
    max_idx = sim_list.index(max(sim_list))
    
    final_path = 'User/'+str(max_idx+1)
    ref = db.reference(path)  # 데이터를 가져올 경로 지정
    #2 나와야 정상
    #2는 index니까 db에서는 User 3 데이터 가꼬오면 된다!
    final_data = ref.get()
    print(final_data)
    
    
    return final_data

if __name__ == '__main__':
    
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


1/1 [==============================] - 1s 935ms/step
3
<class 'str'>
불러옴
추가함
<class 'str'>
불러옴
추가함
<class 'str'>
불러옴
추가함


127.0.0.1 - - [22/May/2023 19:58:35] "GET / HTTP/1.1" 200 -


{'Test2': '7.449025840600143e-09 5.728312402197844e-08 1.857817366612835e-08 3.5678906495917317e-09 1.1032516056275199e-07 8.879852941845456e-08 9.447358451097898e-09 4.110435014581526e-08 1.7154357578874624e-07 7.83342013477295e-09 7.761630449465429e-09 1.5465698055550092e-08 1.541718930297975e-08 3.5382619056889553e-09 1.9613857205058594e-08 2.0909796116796997e-09 1.8347754426883967e-09 7.582718097864927e-08 1.6059097163179104e-08 3.250257663012235e-08 1.259486879234828e-07 6.249517348067002e-09 1.741939854582597e-07 1.5264409967130632e-06 1.6297042293444974e-07 6.727802315253939e-08 1.2261309301209167e-08 2.3340017918371814e-08 2.5127175717898353e-07 1.1685823153584352e-07 4.500585681910252e-09 1.6107499334339082e-07 5.01921590512211e-07 5.1737963246978325e-08 8.344806019522366e-07 3.0323597854930995e-08 4.327081910560082e-07 2.706921264916673e-08 3.465457325546595e-07 3.94389196856082e-08 1.3215978533764883e-08 1.3349644945037653e-08 5.926595392224954e-08 1.599954124742453e-08 1.27